# Basic Retrieval Model

Retrieval models are often composed of two sub-models:
* A query model computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
* A candidate model computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

<div>
  <center>
    <img src='../Img/retrieval_model.PNG' width='50%' style='display:block' title='retrieval model'>
  </center>
</div>

We're going to:

* Get our data and split it into a training and test set.
* Implement a retrieval model.
* Fit and evaluate it.
* Export it for efficient serving by building an approximate nearest neighbours (ANN) index.

## The dataset
[[Link]](https://grouplens.org/datasets/movielens/)

The Movielens dataset is a classic dataset from the GroupLens research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

* It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of implicit feedback, where users' watches tell us which things they prefer to see and which they'd rather not see.
* It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of explicit feedback: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.

In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.

## Imports

In [2]:
import os
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm

import tensorflow as tf
import tensorflow_recommenders as tfrs

# tensorboard needs
from tensorboard import notebook
import time
import subprocess
import platform
import os
import re

# utils
from utils import time_utils
from utils.display_dataframe import grid_df_display
from IPython.display import display, HTML

## Preparing the dataset

In [3]:
# Read the data
user_name = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
user_df = pd.read_csv('../Data/ml-100k/u.user', sep='|', names=user_name)

ratings_name = ['user_id', 'movie_id', 'user_rating', 'timestamp']
ratings_df = pd.read_csv('../Data/ml-100k/u.data', sep='\t', names=ratings_name)

movies_name = ['movie_id', 'movie_title', 'release_data', 'video_release_date',
               'IMDb_URL', 'unknow', 'Action', 'Adventure', 'Animation', 'Children’s',
               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
               'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
               'War', 'Western']
movies_df = pd.read_csv('../Data/ml-100k/u.item', sep='|', encoding='ISO-8859-1', header=None, names=movies_name)

In [4]:
grid_df_display(
    list_df = [user_df, ratings_df, movies_df],
    list_df_name = ['user_df', 'ratings_df', 'movies_df'],
    list_number_of_data = [5, 5, 5],
    row=2, col=2, fill='row'
)

In [5]:
ratings_df = pd.merge(ratings_df, movies_df, on='movie_id', how='left')

In [6]:
# you must to drop columns you don't need. Otherwise, the next cell will get error
ratings_df = ratings_df[['movie_title', 'user_id']]
ratings_df = ratings_df.astype({'user_id': str})
movies_df = movies_df[['movie_title']]

In [7]:
# Select the basic features.
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df)).map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})

In [8]:
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df)).map(lambda x: x['movie_title'])

To fit and evaluate the model, we need to split it into a training and evaluation set. In an industrial recommender system, this would most likely be done by time: the data up to time $T$ would be used to predict interactions after $T$.

In [9]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie titles present in the data.

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [10]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x['user_id'])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## Implementing a model
Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The query tower
The first step is to decide on the dimensionality of the query and candidate representations:

In [11]:
"""
Higher values will correspond to models that may be more accurate
but will also be slower to fit and more prone to overfitting.
"""
embedding_dimension = 32

The second is to define the model itself. Here, we're going to use Keras preprocessing layers to first convert user ids to integers, and then convert those to user embeddings via an `Embedding` layer. Note that we use the list of unique user ids we computed earlier as a vocabulary:

In [12]:
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
        # We add an additional embedding to account for unknown tokens.
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

A simple model like this corresponds exactly to a classic [matrix factorization](https://ieeexplore.ieee.org/abstract/document/4781121) approach. While defining a subclass of tf.keras.Model for this simple model might be overkill, we can easily extend it to an arbitrarily complex model using standard Keras components, as long as we return an embedding_dimension-wide output at the end.

### The candidate tower

In [13]:
item_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metrics
In our training data we have positive (user, movie) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the `tfrs.metrics.FactorizedTopK` metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation. It help us to understand how often the true candidate is in the top K candidates for a given query.

In our case, that's the movies dataset, converted into embeddings via our movie model:

In [14]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128).map(item_model)
)

### Loss
The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the Retrieval task object: a convenience wrapper that bundles together the loss function and metric computation:

In [15]:
task = tfrs.tasks.Retrieval(
    metrics=metrics
)

**The task itself is a Keras layer that takes the query and candidate embeddings as arguments, and returns the computed loss: we'll use that to implement the model's training loop.**

### The full model
We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss method`, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [16]:
class BasicRetrivalModel(tfrs.Model):

    def __init__(self, user_model, item_model):
        super().__init__()
        self.item_model: tf.keras.Model = item_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_item_embeddings = self.item_model(features["movie_title"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_item_embeddings)

The `tfrs.Model` base class is a simply convenience class: it allows us to compute both training and test losses using the same method.

Under the hood, it's still a plain Keras model. You could achieve the same functionality by inheriting from `tf.keras.Model` and overriding the train_step and `test_step` functions (see [the guide](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) for details):

In [17]:
# class NoBaseClassBasicRetrivalModel(tf.keras.Model):

#     def __init__(self, user_model, item_model):
#         super().__init__()
#         self.item_model: tf.keras.Model = item_model
#         self.user_model: tf.keras.Model = user_model
#         self.task: tf.keras.layers.Layer = task

#     def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        
#         # Set up a gradient tape to record gradients.
#         with tf.GradientTape() as tape:
#             # Loss computation.
#             user_embeddings = self.user_model(features["user_id"])
#             positive_movie_embeddings = self.movie_model(features["movie_title"])
#             loss = self.task(user_embeddings, positive_movie_embeddings)

#             # Handle regularization losses as well.
#             regularization_loss = sum(self.losses)

#             total_loss = loss + regularization_loss

#         gradients = tape.gradient(total_loss, self.trainable_variables)
#         self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

#         metrics = {metric.name: metric.result() for metric in self.metrics}
#         metrics["loss"] = loss
#         metrics["regularization_loss"] = regularization_loss
#         metrics["total_loss"] = total_loss

#         return metrics

#     def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        
#         # Loss computation.
#         user_embeddings = self.user_model(features["user_id"])
#         positive_item_embeddings = self.item_model(features["movie_title"])
#         loss = self.task(user_embeddings, positive_item_embeddings)

#         # Handle regularization losses as well.
#         regularization_loss = sum(self.losses)

#         total_loss = loss + regularization_loss

#         metrics = {metric.name: metric.result() for metric in self.metrics}
#         metrics["loss"] = loss
#         metrics["regularization_loss"] = regularization_loss
#         metrics["total_loss"] = total_loss

#         return metrics

Then shuffle, batch, and cache the training and evaluation data.

In [18]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the model:<br>
If you want to monitor the training process with TensorBoard, you can add a TensorBoard callback to fit() function and then start TensorBoard using `%tensorboard --logdir logs/fit`. Please refer to [TensorBoard documentation](https://www.tensorflow.org/tensorboard/get_started) for more details.

In [19]:
def train_model():
    """
    After defining the model, we can use standard Keras fitting and
    evaluation routines to fit and evaluate the model.
    """
    # Let's first instantiate the model.
    model = BasicRetrivalModel(user_model, item_model)
    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
    
    # create log directory
    logdir = os.path.join('../Log/retrieval/fit', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    
    # Then train the model
    model.fit(cached_train,
              epochs=5,
              callbacks=[tensorboard_callback])
    return model

In [20]:
model = train_model()

Epoch 1/5
10/10 [==============================] - 27s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top_k/top_10_categorical_accuracy: 0.0200 - factorized_top_k/top_50_categorical_accuracy: 0.1002 - factorized_top_k/top_100_categorical_accuracy: 0.1765 - loss: 69898.1655 - regularization_loss: 0.0000e+00 - total_loss: 69898.1655
Epoch 2/5
10/10 [==============================] - 23s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0034 - factorized_top_k/top_5_categorical_accuracy: 0.0200 - factorized_top_k/top_10_categorical_accuracy: 0.0398 - factorized_top_k/top_50_categorical_accuracy: 0.1747 - factorized_top_k/top_100_categorical_accuracy: 0.2991 - loss: 67516.5582 - regularization_loss: 0.0000e+00 - total_loss: 67516.5582
Epoch 3/5
10/10 [==============================] - 22s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0038 - factorized_top_k/top_5_categorical_accuracy: 0.0237

## TensorBoard

In [21]:
notebook.list()  # View open Tensorboard instances

Known TensorBoard instances:
  - port 6006: logdir ../Log/fit (started 0:30:58 ago; pid 3908)
  - port 6006: logdir ../Log/fit (started 0:10:11 ago; pid 9540)


In [22]:
def process_exists(process_name):
    """
    check if process currently exists in OS System Taklist
    """
    MY_PLATFORM = platform.system()
    if MY_PLATFORM == 'Windows':
        call = 'TASKLIST /FI "IMAGENAME eq ' + process_name + '"'
        run_obj = subprocess.run(call, capture_output=True)
        if re.search(process_name,
                    run_obj.stdout.decode('utf-8', 'backslashreplace')):
            return True
        else:
            return False
    else:
        p = subprocess.Popen(['ps', '-A'], stdout=subprocess.PIPE)
        out, err = p.communicate()
        out = out.decode('utf-8', 'backslashreplace')
        for line in out.splitlines():
            if process_name in line:
                return True
        return False
    
logs_base_dir = '../Log/retrieval/fit/'

if process_exists('tensorboard.exe'):
    pass
elif process_exists('tensorboard'):
    pass
else:
    print('launch tensorboard process...')
    popen_obj = subprocess.Popen(['tensorboard', '--logdir', logs_base_dir, '--port', '6006'])
    
    # patch to wait process
    time.sleep(30)
    
# Load the TensoorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir {logs_base_dir} --port 6006

launch tensorboard process...


Reusing TensorBoard on port 6006 (pid 15136), started 0:00:20 ago. (Use '!kill 15136' to kill it.)

In [23]:
# # Load the TensorBoard notebook extension
# %load_ext tensorboard

In [24]:
# %tensorboard --logdir ../Log/fit --port 6006

As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

Note that, in this example, we evaluate the metrics during training as well as evaluation. Because this can be quite slow with large candidate sets, it may be prudent to turn metric calculation off in training, and only run it in evaluation.

## Model Evaluation
Finally, we can evaluate our model on the test set:

In [25]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 5s 806ms/step - factorized_top_k/top_1_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0145 - factorized_top_k/top_50_categorical_accuracy: 0.1080 - factorized_top_k/top_100_categorical_accuracy: 0.2149 - loss: 31190.7321 - regularization_loss: 0.0000e+00 - total_loss: 31190.7321


{'factorized_top_k/top_1_categorical_accuracy': 0.0003499999875202775,
 'factorized_top_k/top_5_categorical_accuracy': 0.0048500001430511475,
 'factorized_top_k/top_10_categorical_accuracy': 0.014499999582767487,
 'factorized_top_k/top_50_categorical_accuracy': 0.10795000195503235,
 'factorized_top_k/top_100_categorical_accuracy': 0.21494999527931213,
 'loss': 28386.7109375,
 'regularization_loss': 0,
 'total_loss': 28386.7109375}

Test set performance is much worse than training performance. This is due to two factors:

1. Our model is likely to perform better on the data that it has seen, simply because it can memorize it. This overfitting phenomenon is especially strong when models have many parameters. It can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data.
2. The model is re-recommending some of users’ already watched movies. These known-positive watches can crowd out test movies out of top K recommendations.

The second phenomenon can be tackled by excluding previously seen movies from test recommendations. This approach is relatively common in the recommender systems literature, but we don’t follow it in these tutorials. If not recommending past watches is important, we should expect appropriately specified models to learn this behaviour automatically from past user history and contextual information. Additionally, it is often appropriate to recommend the same item multiple times (say, an evergreen TV series or a regularly purchased item).

## Making predictions
Now that we have a model, we would like to be able to make predictions. We can use the `tfrs.layers.factorized_top_k.BruteForce` layer to do this.

In [26]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=20)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.item_model)))
)

# Get recommendations.
score, items = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {items[0, :10]}")

Recommendations for user 42: [b'All Dogs Go to Heaven 2 (1996)' b'Father of the Bride Part II (1995)'
 b'Grumpier Old Men (1995)' b'Rudy (1993)' b'Outbreak (1995)'
 b'Rent-a-Kid (1995)' b'Michael (1996)' b'Firm, The (1993)'
 b'Indian in the Cupboard, The (1995)'
 b'Bridges of Madison County, The (1995)']


In [27]:
recommendation_list = items.numpy()[0]
recommendation_list = [i.decode('UTF-8') for i in recommendation_list]
recommendation_list

['All Dogs Go to Heaven 2 (1996)',
 'Father of the Bride Part II (1995)',
 'Grumpier Old Men (1995)',
 'Rudy (1993)',
 'Outbreak (1995)',
 'Rent-a-Kid (1995)',
 'Michael (1996)',
 'Firm, The (1993)',
 'Indian in the Cupboard, The (1995)',
 'Bridges of Madison County, The (1995)',
 'Tombstone (1993)',
 'Just Cause (1995)',
 'Pretty Woman (1990)',
 'Boys on the Side (1995)',
 'Man Without a Face, The (1993)',
 'Lion King, The (1994)',
 'Foxfire (1996)',
 'Something to Talk About (1995)',
 'Land Before Time III: The Time of the Great Giving (1995) (V)',
 'Ghost (1990)']

In [28]:
li_all = []
for _id in tqdm(ratings_df['user_id'].unique()):
    scores, items = index(tf.constant([_id]))
    
    recommendation_list = items.numpy()[0]
    recommendation_list = [item.decode('UTF-8') for item in recommendation_list]
    
    recommendation_score_list = scores.numpy()[0]
    recommendation_score_list = [score for score in recommendation_score_list]
    
    li_all.append([_id, recommendation_list, recommendation_score_list])

result_df = pd.DataFrame(li_all, columns=['user_id', 'item', 'score'])

  0%|          | 0/943 [00:00<?, ?it/s]

In [29]:
result_df.head()

,user_id,item,score
0,196,"[Marvin's Room (1996), Muriel's Wedding (1994)...","[3.2124872, 3.2012463, 3.050365, 2.9061198, 2...."
1,186,"[Rich Man's Wife, The (1996), Set It Off (1996...","[3.7101855, 3.312335, 3.2354455, 3.2006693, 3...."
2,22,"[Houseguest (1994), Jingle All the Way (1996),...","[3.1025152, 3.0470707, 2.909378, 2.8906102, 2...."
3,244,"[Dazed and Confused (1993), Now and Then (1995...","[2.5598068, 2.5264373, 2.1983883, 2.1912973, 2..."
4,166,"[Flubber (1997), Mouse Hunt (1997), Anastasia ...","[5.895788, 5.4204464, 5.307049, 5.2902536, 5.2..."


In [30]:
timestamp_prefix = time_utils._timestamp_pretty()
result_df.to_pickle(f'../Output/recommend_result_{timestamp_prefix}.pkl')

In [31]:
pd.read_pickle(f'../Output/recommend_result_{timestamp_prefix}.pkl')

,user_id,item,score
0,196,"[Marvin's Room (1996), Muriel's Wedding (1994)...","[3.2124872, 3.2012463, 3.050365, 2.9061198, 2...."
1,186,"[Rich Man's Wife, The (1996), Set It Off (1996...","[3.7101855, 3.312335, 3.2354455, 3.2006693, 3...."
2,22,"[Houseguest (1994), Jingle All the Way (1996),...","[3.1025152, 3.0470707, 2.909378, 2.8906102, 2...."
3,244,"[Dazed and Confused (1993), Now and Then (1995...","[2.5598068, 2.5264373, 2.1983883, 2.1912973, 2..."
4,166,"[Flubber (1997), Mouse Hunt (1997), Anastasia ...","[5.895788, 5.4204464, 5.307049, 5.2902536, 5.2..."
...,...,...,...
938,939,"[Associate, The (1996), That Old Feeling (1997...","[4.7164116, 4.611521, 4.491795, 4.443122, 4.20..."
939,936,"[Cement Garden, The (1993), Hotel de Love (199...","[4.633749, 4.630556, 4.4191675, 4.2371416, 4.1..."
940,930,"[Emma (1996), Postino, Il (1994), Addicted to ...","[3.1928847, 3.1273036, 2.8424704, 2.8422384, 2..."
941,920,"[Edge, The (1997), Palmetto (1998), Devil's Ad...","[5.2678165, 5.120018, 5.011171, 5.0096664, 4.9..."


## Model serving
After the model is trained, we need a way to deploy it.

In a two-tower retrieval model, serving has two components:

* a serving query model, taking in features of the query and transforming them into a query embedding, and
* a serving candidate model. This most often takes the form of an approximate nearest neighbours (ANN) index which allows fast approximate lookup of candidates in response to a query produced by the query model.

In TFRS, both components can be packaged into a single exportable model, giving us a model that takes the raw user id and returns the titles of top movies for that user. This is done via exporting the model to a `SavedModel` format, which makes it possible to serve using [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving).

To deploy a model like this, we simply export the `BruteForce` layer we created above:

In [32]:
# # Export the query model.
# with tempfile.Tempporary() as tmp:
#     path = os.path.join(tmp, 'model')
    
#     # Save the index.
#     tf.saved_model.save(index, path)
    
#     # Load it back; can alseo be done in TensorFlow Serving.
#     loaded = tf.saved_model.load(path)
    
#     # Pass a user id in, get top predicted movie titles back.
#     scores, titles = loaded(['42'])  # give the user name for query recommendations
    
#     print(f'Recommendations: {titles[0][:3]}')

In [33]:
tf.saved_model.save(index, f'../Model/basic_retrival_model/{timestamp_prefix}')

INFO:tensorflow:Assets written to: ../Model/basic_retrival_model/202204151241\assets


INFO:tensorflow:Assets written to: ../Model/basic_retrival_model/202204151241\assets


In [34]:
loaded = tf.saved_model.load(f'../Model/basic_retrival_model/{timestamp_prefix}')

In [35]:
scores, titles = loaded(['42'])
print(f'Recommendations: {titles[0][:3]}')

Recommendations: [b'All Dogs Go to Heaven 2 (1996)' b'Father of the Bride Part II (1995)'
 b'Grumpier Old Men (1995)']


In [36]:
scores, titles = loaded(['66'])
print(f'Recommendations: {titles[0][:3]}')

Recommendations: [b'Con Air (1997)' b'Face/Off (1997)' b'Toy Story (1995)']


## More Advanced Skill to Serving Model

We can also export an approximate retrieval index to speed up predictions. This will make it possible to efficiently surface recommendations from sets of tens of millions of candidates.

To do so, we can use the `scann` package. This is an optional dependency of TFRS, and we installed it separately at the beginning of this tutorial by calling `!pip install -q scann`.

In [37]:
# scann_index = tfrs.layers.factorized_top_k.ScaNN(model.user_model)
# scann_index.index_from_dataset(
#     tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
# )

**This layer will perform approximate lookups: this makes retrieval slightly less accurate, but orders of magnitude faster on large candidate sets.**

In [38]:
# # Get recommendations.
# _, titles = scann_index(tf.constant(["42"]))
# print(f"Recommendations for user 42: {titles[0, :3]}")

Exporting it for serving is as easy as exporting the `BruteForce` layer:

In [39]:
# # Export the query model.
# with tempfile.TemporaryDirectory() as tmp:
#     path = os.path.join(tmp, "model")

#     # Save the index.
#     tf.saved_model.save(
#         index,
#         path,
#         options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
#     )

#     # Load it back; can also be done in TensorFlow Serving.
#     loaded = tf.saved_model.load(path)

#     # Pass a user id in, get top predicted movie titles back.
#     scores, titles = loaded(["42"])

#     print(f"Recommendations: {titles[0][:3]}")

To learn more about using and tuning fast approximate retrieval models, have a look at our [efficient serving](https://www.tensorflow.org/recommenders/examples/efficient_serving) tutorial.

## Item-to-item recommendation
In this model, we created a user-movie model. However, for some applications (for example, product detail pages) it's common to perform item-to-item (for example, movie-to-movie or product-to-product) recommendations.

Training models like this would follow the same pattern as shown in this tutorial, but with different training data. Here, we had a user and a movie tower, and used (user, movie) pairs to train them. In an item-to-item model, we would have two item towers (for the query and candidate item), and train the model using (query item, candidate item) pairs. These could be constructed from clicks on product detail pages.

## Next steps
To expand on what is presented here, have a look at:

1. Learning multi-task models: jointly optimizing for ratings and clicks.
2. Using movie metadata: building a more complex movie model to alleviate cold-start.